In [4]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Load the cleaned dataset
data_cleaned = pd.read_csv('../../../Data_processing/Outputs/train.csv')

# data_cleaned = data_cleaned.loc[data_cleaned['usubjid'].isin(data_cleaned[data_cleaned['label'] == 1]["usubjid"].unique())]
data_cleaned.dropna(inplace=True)   

 #['full', 'diag', 'tied', 'spherical']
# Apply global min-max normalization to the specified columns
# scaler = MinMaxScaler()
# data_cleaned[['aval_AlloMap', 'aval_AlloSure']] = scaler.fit_transform(data_cleaned[['aval_AlloMap', 'aval_AlloSure']])

# def minmax_scaler_by_id(df, columns, id_column):
#     df_scaled = df.copy()
#     scaler = MinMaxScaler()

#     # Apply MinMaxScaler for each group of 'usubjid'
#     df_scaled[columns] = df.groupby(id_column)[columns].apply(lambda x: pd.DataFrame(scaler.fit_transform(x), columns=columns, index=x.index))
    
#     return df_scaled

# # Apply the scaler at the ID level
# data_cleaned = minmax_scaler_by_id(data_cleaned, ['aval_AlloMap', 'aval_AlloSure', "ady_dna"], 'usubjid')

# Group data by 'usubjid' to maintain sequence structure
grouped_data = data_cleaned.groupby('usubjid')

# Shuffle
shuffled_df = grouped_data.sample(frac=1, random_state=42)
shuffled_df = shuffled_df.sort_values(['usubjid', 'ady_dna'])

# Reset the index of the shuffled dataframe
grouped_data = shuffled_df.reset_index(drop=True)
grouped_data = grouped_data.groupby('usubjid')

X_grouped = [group[['aval_AlloSure', 'aval_AlloMap', "ady_dna"]].values for name, group in grouped_data]
y_grouped = [group['label'].values for name, group in grouped_data]


# Calculate the split index for 80%/20%
split_index = int(0.80 * len(X_grouped))

# Split the sequences into training and testing sets
X_train_grouped = X_grouped[:split_index]
X_test_grouped = X_grouped[split_index:]
y_train_grouped = y_grouped[:split_index]
y_test_grouped = y_grouped[split_index:]

# Concatenate the sequences to form the training and testing sets
X_train = np.concatenate(X_train_grouped)
X_test = np.concatenate(X_test_grouped)
y_train = np.concatenate(y_train_grouped)
y_test = np.concatenate(y_test_grouped)

# Prepare lengths for the sequences
train_lengths = [len(x) for x in X_train_grouped]
test_lengths = [len(x) for x in X_test_grouped]

# Initialize and train the HMM model
model = hmm.GaussianHMM(n_components=2, covariance_type="tied", n_iter=50000, random_state=42, init_params='stmc')


model.fit(X_train, lengths=train_lengths)

# Evaluate the model on the test set
log_likelihood = model.score(X_test, lengths=test_lengths)
print(f"Log-Likelihood on Test Set: {log_likelihood}")

# Predict the hidden states for the testing set
predicted_hidden_states = model.predict(X_test, lengths=test_lengths)

# Remove NaNs from y_test for proper evaluation
y_test_clean = y_test[~np.isnan(y_test)]
predicted_hidden_states_clean = predicted_hidden_states[~np.isnan(y_test)]

# Generate and print classification metrics
print("Classification Report:")
print(classification_report(y_test_clean, predicted_hidden_states_clean))

print("Confusion Matrix:")
print(confusion_matrix(y_test_clean, predicted_hidden_states_clean))

# If it's a binary classification, you can also calculate AUC-ROC
if len(np.unique(y_test_clean)) == 2:
    auc = roc_auc_score(y_test_clean, predicted_hidden_states_clean)
    print(f"AUC-ROC: {auc}")

Log-Likelihood on Test Set: -24514.15051202856
Classification Report:
              precision    recall  f1-score   support

         0.0       0.93      0.92      0.92      2114
         1.0       0.20      0.24      0.22       186

    accuracy                           0.86      2300
   macro avg       0.57      0.58      0.57      2300
weighted avg       0.87      0.86      0.87      2300

Confusion Matrix:
[[1937  177]
 [ 142   44]]
AUC-ROC: 0.576415804518774
